## HR detector on the output from compoundRay

- The ommatidia data contains the lens positions, ommatidia directions, and acceptance angles. The hexagonal coordinates detemines the neighboring relationship of the ommatidia. (see "hex_coordinates.ipynb")

- Input data from compoundRay is a time sequence of vectors. Each vector contains the averaged values of all ommatidia. The order of the vector corresponds to the order of the ommatidia data. 

- The main idea is to compare neighboring ommatidia values at consecutive time point to compute motion. The time delay can be modeled as a low pass filter (see "lowpass_eg.ipynb"). So one way (what we will do here) to do this is to low-pass filter the ommatidia values (add time delay), and then compare (multiple) with adjacent ommatidium values (no filter). Another way is to compare values from different times. 


1. First apply a low-pass filter to single-ommatidia values 

2. Compare filtered values to original ones from adjacent ommatidia. This is doen separately in all 6 directions (see "grid_nb.ipynb"). This is essentially HR detector. 

3. Low-pass filter the output again to make it smooth. 

4. Note that left and right eyes shold be done separately, therefore need separate compoundRay simulations.


In [2]:
import numpy as np
from cmath import pi

### parameters following Michael's "JEB2013 sims" code

Assume fly speed is 23 cm/s.

distance step size = 0.05 cm

time step size = 0.05/23 s

Low pass filter for ommatidium = 30 ms

Low pass filter for output signal = 0.2 s

Low pass filter for ommatidium (RC time): tau_HR = 30 ms = $\tau_{HR}$  
$A_{i} = 1 - \frac{2 \tau_{HR}}{T}$  
$B_{i} = 1 + \frac{2 \tau_{HR}}{T}$  

Low pass filter for output signal (RC time): tau_out = 0.2 s = $\tau_{out}$  
$A_{o} = 1 - \frac{2 \tau_{out}}{T}$  
$B_{o} = 1 + \frac{2 \tau_{out}}{T}$  

Let $u(t)$ be the input (raw ommatidia value) at time t, and $y(t)$ be the output (after lowpass filter) at time t. Then  
$y(t) = \frac{u(t) + u(t-1) - A y(t-1)}{B}$

Now we compare the adjacent values before and after the low-pass filter (time delay) compute motion signal $M_{HR}$:  
$M_{HR} = y(t, x0) \times u(t, x1) - y(t, x1) \times u(t, x1)$